In [1]:
import pandas as pd
import numpy as np
import os
import pyproj
from pyproj import Transformer
import string
import os
import requests, zipfile, io

In [2]:
# Data Directory / Files
data_dir = "./DOI-WGMS-FoG-2019-12"
data_archive_url = "https://wgms.ch/downloads/DOI-WGMS-FoG-2019-12.zip"
target_path = './DOI-WGMS-FoG-2019-12.zip'

# Download if not present,      
if not os.path.exists(data_dir):
    response = requests.get(data_archive_url, stream=True)
    with open(target_path, "wb") as target_file:
        for chunk in response.iter_content(chunk_size=512):
            if chunk:  
                target_file.write(chunk)
    z = zipfile.ZipFile(target_path)
    z.extractall()
    


In [3]:
# WGMS Data Files
a_glacier_file = "WGMS-FoG-2019-12-A-GLACIER.csv"
b_glacier_file = "WGMS-FoG-2019-12-B-STATE.csv"
d_change_file = "WGMS-FoG-2019-12-D-CHANGE.csv"
e_massbalance_file = "WGMS-FoG-2019-12-E-MASS-BALANCE-OVERVIEW.csv"
ee_massbalance_file = "WGMS-FoG-2019-12-EE-MASS-BALANCE.csv"

#### At this point, open the `a_glacier_file`,  `ee_glacier_file` files and resave in utf-8 format (LibreOffice Calc works).

In [4]:
# Main dataframe containing overall information
df_compiled = pd.DataFrame()

### Extract relevant Glacial Characteristics from the WGMS_A file

In [5]:
df_A = pd.read_csv(os.path.join(data_dir, a_glacier_file))
df_A.dropna(axis="rows", subset=["LONGITUDE", "LATITUDE"], inplace=True)

In [6]:
df_A.columns

Index(['POLITICAL_UNIT', 'NAME', 'WGMS_ID', 'GEN_LOCATION', 'SPEC_LOCATION',
       'LATITUDE', 'LONGITUDE', 'PRIM_CLASSIFIC', 'FORM', 'FRONTAL_CHARS',
       'EXPOS_ACC_AREA', 'EXPOS_ABL_AREA', 'PARENT_GLACIER', 'REMARKS',
       'GLACIER_REGION_CODE', 'GLACIER_SUBREGION_CODE'],
      dtype='object')

In [7]:
# Prettify Capitalization
df_A['NAME'] = df_A['NAME'].apply(lambda x: string.capwords(x))

notna = df_A['SPEC_LOCATION'].notna()
df_A.loc[notna,'SPEC_LOCATION'] = df_A.loc[notna,'SPEC_LOCATION'].apply(lambda x: string.capwords(str(x)))

# Change to Float for Consistency
df_A['PRIM_CLASSIFIC'] = df_A['PRIM_CLASSIFIC'].astype(float)
df_A['FORM'] = df_A['FORM'].replace(' ', np.nan).astype(float)
df_A['FRONTAL_CHARS'] = df_A['FRONTAL_CHARS'].astype(float)






In [8]:
# Extract relevant columns
A_columns = [
    "WGMS_ID",
    "LONGITUDE",
    "LATITUDE",
    "POLITICAL_UNIT",
    "GLACIER_REGION_CODE",
    "SPEC_LOCATION",
    "NAME",
    "PRIM_CLASSIFIC",
    "FORM",
    "FRONTAL_CHARS",
    "EXPOS_ACC_AREA",
    "EXPOS_ABL_AREA",
    "REMARKS",
]

df_compiled = df_A.loc[:, A_columns]

### Extract additional data from WGMS_B File

In [9]:
df_B = pd.read_csv(os.path.join(data_dir, b_glacier_file))
df_B = df_B.query("YEAR > 0")
df_B.columns

Index(['POLITICAL_UNIT', 'NAME', 'WGMS_ID', 'YEAR', 'HIGHEST_ELEVATION',
       'MEDIAN_ELEVATION', 'LOWEST_ELEVATION', 'ELEVATION_UNC', 'LENGTH',
       'LENGTH_UNC', 'AREA', 'AREA_UNC', 'SURVEY_DATE',
       'SURVEY_PLATFORM_METHOD', 'INVESTIGATOR', 'SPONS_AGENCY', 'REFERENCE',
       'REMARKS', 'PUB_IN_FOG', 'PUB_IN_GGCB'],
      dtype='object')

In [10]:
df_B_reduced = df_B.loc[
    :,
    [
        "WGMS_ID",
        "YEAR",
        "HIGHEST_ELEVATION",
        "LOWEST_ELEVATION",
        "INVESTIGATOR",
        "SPONS_AGENCY",
        "REFERENCE",
    ],
]
df_B_reduced.drop_duplicates("WGMS_ID", keep="last", inplace=True)

### Time Series

Create time series for the following data:
1. Thickness Change
2. Mass Balance
3. Length

In [11]:
def ts_helper(df, columns):
    """Extract time series data so that: 
        1: There is one measurement per year; multiple measurements are summarized crudely with median()
        """

    # One measurement per Year
    df_median = df.loc[:, columns].groupby(columns[0:2], as_index=False).median()
    df_out = df_median

    return df_out

### Thickness Change

In [12]:
df_D = pd.read_csv(os.path.join(data_dir, d_change_file))
df_D.columns

Index(['POLITICAL_UNIT', 'NAME', 'SURVEY_ID', 'WGMS_ID', 'YEAR', 'LOWER_BOUND',
       'UPPER_BOUND', 'AREA_SURVEY_YEAR', 'AREA_CHANGE', 'AREA_CHANGE_UNC',
       'THICKNESS_CHG', 'THICKNESS_CHG_UNC', 'VOLUME_CHANGE',
       'VOLUME_CHANGE_UNC', 'SURVEY_DATE', 'SD_PLATFORM_METHOD',
       'REFERENCE_DATE', 'RD_PLATFORM_METHOD', 'INVESTIGATOR', 'SPONS_AGENCY',
       'REFERENCE', 'REMARKS', 'PUB_IN_FOG', 'PUB_IN_GGCB', 'REF_ID'],
      dtype='object')

In [13]:
th_columns = ["WGMS_ID", "YEAR", "THICKNESS_CHG",'REFERENCE_DATE']
df_D_ = df_D.loc[:,th_columns]
df_D_.dropna(axis=0, how="any", inplace=True)

In [14]:
df_D_['REFERENCE_DATE'] = df_D_['REFERENCE_DATE'].apply(lambda x: int(str(x)[0:4]))

In [15]:
df_thickness_chg = ts_helper(df_D_, th_columns)

### Area

In [16]:
area_columns = ["WGMS_ID", "YEAR", "AREA"]
df_area = ts_helper(df_B, area_columns)
df_area.dropna(axis="rows", inplace=True)

In [17]:
df_area.head(n=5)

,WGMS_ID,YEAR,AREA
0,0,1959,38.90
1,0,1975,38.90
2,0,2014,38.54
4,1,1975,0.63
5,1,2005,0.61


### Length

In [18]:
length_columns = ["WGMS_ID", "YEAR", "LENGTH"]
df_length = ts_helper(df_B, length_columns)
df_length.dropna(axis="rows", inplace=True)

In [19]:
df_length.head(n=5)

,WGMS_ID,YEAR,LENGTH
0,0,1959,15.4
1,0,1975,15.4
2,0,2014,14.0
3,1,1960,1.4
4,1,1975,1.4


### Mass Balance

In [20]:
df_EE = pd.read_csv(os.path.join(data_dir, ee_massbalance_file))
df_EE.dropna(axis="rows", subset=["ANNUAL_BALANCE"], inplace=True)

EE_columns = ["WGMS_ID", "YEAR", "ANNUAL_BALANCE"]
df_mass_balance = ts_helper(df_EE, EE_columns)

### Extract site Time Series Characteristics

In [21]:
df_wgms = df_compiled.loc[:, ["WGMS_ID"]]

### First Measurement

In [22]:
dfs = [
    df.loc[:, ["WGMS_ID", "YEAR"]].set_index("WGMS_ID")
    for df in [df_thickness_chg, df_mass_balance, df_length, df_area]
]
dfs_ = [df.groupby("WGMS_ID")["YEAR"].min() for df in dfs]
df_first_measurement = pd.concat(dfs_, axis=1, join="outer").min(axis=1).reset_index()
df_first_measurement.rename({0: "FIRST_MEAS"}, axis=1, inplace=True)

### Years of Measurements

In [23]:
dfs = [
    df.loc[:, ["WGMS_ID", "YEAR"]]
    for df in [df_thickness_chg, df_mass_balance, df_length, df_area]
]
dfs_ = pd.concat(dfs).drop_duplicates(keep="first")
df_year_measurement = dfs_.groupby("WGMS_ID").size().reset_index()
df_year_measurement.rename({0: "YEAR_MEASUREMENTS"}, axis=1, inplace=True)

### Which time series data each glacier has

In [24]:
df_thickness_ts_bool = pd.DataFrame({'WGMS_ID': df_thickness_chg['WGMS_ID'].unique(), 'THICKNESS_CHANGE_TS': True})
df_length_ts_bool = pd.DataFrame({'WGMS_ID': df_length['WGMS_ID'].unique(), 'LENGTH_TS': True})
df_area_ts_bool = pd.DataFrame({'WGMS_ID': df_area['WGMS_ID'].unique(), 'AREA_TS': True})
df_mb_ts_bool = pd.DataFrame({'WGMS_ID': df_mass_balance['WGMS_ID'].unique(), 't': True})


In [25]:
df_mb_ts_bool.head(n=5)

,WGMS_ID,MASS_BALANCE_TS
0,0,True
1,1,True
2,16,True
3,24,True
4,30,True


### Concatenate

In [26]:
df_compiled = df_compiled.merge(df_B_reduced, how="left", on="WGMS_ID", validate="1:1")
df_compiled = df_compiled.merge(df_first_measurement, on="WGMS_ID", how="left")
df_compiled = df_compiled.merge(df_year_measurement, on="WGMS_ID", how="left")

df_compiled = df_compiled.merge(df_thickness_ts_bool, on="WGMS_ID", how="left")
df_compiled = df_compiled.merge(df_length_ts_bool, on="WGMS_ID", how="left")
df_compiled = df_compiled.merge(df_area_ts_bool, on="WGMS_ID", how="left")
df_compiled = df_compiled.merge(df_mb_ts_bool, on="WGMS_ID", how="left")


#Set Sensible Null Values
# Set first measurement to 2020 if value is Nan, measured years to zero if value is NaN
df_compiled.replace(
    {
        "FIRST_MEAS": {np.nan: 2020},
        "YEAR_MEASUREMENTS": {np.nan: 0},
        "PRIM_CLASSIFIC": {np.nan: 10},
        "FORM": {np.nan: 10},
        "FRONTAL_CHARS": {np.nan: 10},
        "SPEC_LOCATION": {np.nan: "N/A"},
        "NAME": {np.nan: "N/A"},
        "INVESTIGATOR": {np.nan: "N/A"},
        "SPONS_AGENCY": {np.nan: "N/A"},
        "REMARKS": {np.nan: "N/A"},
        "REFERENCE": {np.nan: "N/A"},
        "THICKNESS_CHANGE_TS": {np.nan: False},
        "LENGTH_TS": {np.nan: False},
        "AREA_TS": {np.nan: False},
        "MASS_BALANCE_TS": {np.nan: False},

    },
    inplace=True,
)

In [27]:
df_compiled.head(n=4)

,WGMS_ID,LONGITUDE,LATITUDE,POLITICAL_UNIT,GLACIER_REGION_CODE,SPEC_LOCATION,NAME,PRIM_CLASSIFIC,FORM,FRONTAL_CHARS,...,LOWEST_ELEVATION,INVESTIGATOR,SPONS_AGENCY,REFERENCE,FIRST_MEAS,YEAR_MEASUREMENTS,THICKNESS_CHANGE_TS,LENGTH_TS,AREA_TS,MASS_BALANCE_TS
0,3628,73.23500,37.10000,AF,ASC,Upper Issik Valley,Northern Issik,10.0,10.0,10.0,...,NaN,N/A,N/A,N/A,2020.0,0.0,False,False,False,False
1,10452,70.17000,35.59500,AF,ASW,Chumar Valley,Pir Yakh,6.0,3.0,8.0,...,4400.0,"Abeer Ahmad Sajood, Hedayatullah Arian","Hydrometeorology Department, Geoscience Facult...",N/A,2018.0,1.0,False,True,True,False
2,13308,73.60173,37.28307,AF,ASC,N/A,Unnamed 13308,10.0,10.0,10.0,...,4720.0,N/A,N/A,RGI5.0,2000.0,2.0,True,False,True,False
3,13310,73.61128,37.25005,AF,ASC,N/A,Unnamed 13310,10.0,10.0,10.0,...,4682.0,N/A,N/A,RGI5.0,2000.0,2.0,True,False,True,False


### Save Files

In [28]:
df_compiled.to_pickle("wgms_combined")
df_thickness_chg.to_pickle("wgms_thickness")
df_mass_balance.to_pickle("wgms_massbalance")
df_length.to_pickle("wgms_length")
df_area.to_pickle("wgms_area")

### WGMS ID of MER DE GLACE

In [29]:
df_compiled[df_compiled["NAME"].str.contains("glace", case=False)]

,WGMS_ID,LONGITUDE,LATITUDE,POLITICAL_UNIT,GLACIER_REGION_CODE,SPEC_LOCATION,NAME,PRIM_CLASSIFIC,FORM,FRONTAL_CHARS,...,LOWEST_ELEVATION,INVESTIGATOR,SPONS_AGENCY,REFERENCE,FIRST_MEAS,YEAR_MEASUREMENTS,THICKNESS_CHANGE_TS,LENGTH_TS,AREA_TS,MASS_BALANCE_TS
16777,353,6.93,45.88,FR,CEU,Mont Blanc Area,Mer De Glace,5.0,1.0,9.0,...,1800.0,"D. Six, C.Vincent",CNRS/Grenoble University,"Berthier, E. and C. Vincent. 2012. J. of Glaci...",1968.0,11.0,True,True,True,True


In [39]:
df_compiled.query("THICKNESS_CHANGE_TS in [True, False]").count()

WGMS_ID                32503
LONGITUDE              32503
LATITUDE               32503
POLITICAL_UNIT         32503
GLACIER_REGION_CODE    32503
SPEC_LOCATION          32503
NAME                   32503
PRIM_CLASSIFIC         32503
FORM                   32503
FRONTAL_CHARS          32503
EXPOS_ACC_AREA          2073
EXPOS_ABL_AREA          2080
REMARKS                32503
YEAR                   10377
HIGHEST_ELEVATION       9779
LOWEST_ELEVATION       10029
INVESTIGATOR           32503
SPONS_AGENCY           32503
REFERENCE              32503
FIRST_MEAS             32503
YEAR_MEASUREMENTS      32503
THICKNESS_CHANGE_TS    32503
LENGTH_TS              32503
AREA_TS                32503
MASS_BALANCE_TS        32503
dtype: int64